# IMPORT LIBRARIES
- pandas: Used for data manipulation and analysis, particularly with tabular data like CSV files.
- numpy: Provides tools for numerical computations and handling multi-dimensional arrays efficiently.
- logging: Enables the recording of log messages for debugging, tracking, and monitoring program execution.
- re: Facilitates pattern matching and manipulation using regular expressions in strings.
- os: Provides functions to interact with the operating system, including file and directory management

In [1]:
import pandas as pd
import numpy as ny
import logging
import re
import os

# Step 1: DEFINE FILE PATHS

In [9]:
# Step 1: Define File paths
input_file = "./datasets/records/zhaopin.csv"
intermediate_file_1 = "./datasets/processed/step1/intermediate_1.csv"
intermediate_file_2 = "./datasets/processed/step2/intermediate_2.csv"
intermediate_file_3 = "./datasets/processed/step3/intermediate_3.csv"
intermediate_file_4 = "./datasets/processed/step4/intermediate_4.csv"
intermediate_file_5 = "./datasets/processed/step5/intermediate_5.csv"
intermediate_file_6 = "./datasets/processed/step6/intermediate_6.csv"
intermediate_file_7 = "./datasets/processed/step7/intermediate_7.csv"
intermediate_file_8 = "./datasets/processed/step8/intermediate_8.csv"
intermediate_file_9 = "./datasets/processed/step9/intermediate_9.csv"
intermediate_file_10 = "./datasets/processed/step10/intermediate_10.csv"
garbage_file = "./datasets/garbage/final_garbage_file.csv"
cleaned_file = "./datasets/to-be-ingested/zhaopin-cleaned_output.csv"
log_file = "./datasets/logs/zhaopin_cleaner_script.log"

# Step 2: SET UP LOGGING
- Logging both to the screen and toa log file

In [10]:
# Step 2: Set up logging
# Create a logger
logger = logging.getLogger("script_logger")
logger.setLevel(logging.INFO)  # Set the minimum logging level

# Create a file handler
file_handler = logging.FileHandler(log_file, mode="a")  # Append to 'script.log'
file_handler.setLevel(logging.INFO)

# Create a console handler
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

# Define a common format for both handlers
formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
file_handler.setFormatter(formatter)
console_handler.setFormatter(formatter)

# Add both handlers to the logger
logger.addHandler(file_handler)
logger.addHandler(console_handler)


# Step 3: LOAD INITIAL DATA

In [11]:
# Step 3: Load initial data
df = pd.read_csv(input_file)

C:\Users\max_m\AppData\Local\Temp\ipykernel_28456\2747940852.py:2: DtypeWarning: Columns (0,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_file)


# Step 4: GET STATISTICS
- Checking for raw data lngth and missing values

In [12]:
# Verify the raw data length (before cleaning)
raw_data_length = len(df)
print(f"Raw data length: {raw_data_length}")

# Check for missing values
print("Missing values per column:")
print(df.isnull().sum())

# Fill missing values with the mean of the column
#df.fillna(df.mean(), inplace=True)

Raw data length: 1606289
Missing values per column:
BID             0
EMail         237
姓名          22707
生日          21508
身份证        199640
手机          70736
学历         239897
收入         707454
行业代码       707385
工作时间       196537
字段15          218
hangye    1371663
memo       617709
dtype: int64


# Step 5: PANDAS ACCEPTANCE/NORMALIZE DATASET
- 5a - remove blank rows
- 5b - remove blank columns
- 5c - Remove non-ascii/special characters
- 5d - Encode in UTF-8 format
- 5e - Standardize text/text columns
- 5f - Output to intermediate_file_5

In [13]:
# Step 5: Normalize Dataset

def clean_dataframe(df, output_file):
    # 5a: Removing rows with all NaN values
    print("Removing rows with all NaN values")
    df.dropna(how='all', inplace=True)

    # 5b: Removing rows where all columns are empty or null
    print("Removing rows where all columns are empty or null")
    problematic_rows = df[df.isnull().all(axis=1)].index
    if not problematic_rows.empty:
        df.drop(index=problematic_rows, inplace=True)

    # 5c: Removing non-ASCII characters/special characters
    print("Removubg non-ASCII characters/special characters")
    def remove_non_ascii(text):
        if isinstance(text, str):
            return re.sub(r'[\x80-\xFF]', '', text)  # Removing non-ASCII characters
        return text

    df = df.applymap(remove_non_ascii)

    # 5d: Encode the data using UTF-8 format
    # Pandas automatically handles encoding when writing CSV files, so no explicit action needed here.

    # 5e: Standardizing text/text columns
    print("Standardizing text/text columns")
    def standardize_text(text):
        if isinstance(text, str):
            # Lowercase text
            text = text.lower()
            # strip whitespace
            text = text.strip()
            # Remove non-ASCII characters (but preserve for foreign languages if necessary)
            return re.sub(r'[\x80-\xFF]', '', text)
        return text

    text_columns = df.select_dtypes(include=['object']).columns
    for col in text_columns:
        df[col] = df[col].apply(standardize_text)

    # 5d: Output the data frame to a CSV file
    df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"Cleaned data saved to {output_file}")

clean_dataframe(df, intermediate_file_5)


Removing rows with all NaN values
Removing rows where all columns are empty or null
Removubg non-ASCII characters/special characters


C:\Users\max_m\AppData\Local\Temp\ipykernel_28456\1325827101.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(remove_non_ascii)


Standardizing text/text columns
Cleaned data saved to ./datasets/processed/step5/intermediate_5.csv


# STEP 6: DROPPING AND RENAMING COLUMNS
- 6a: Load data
- 6b: Drop columns
- 6c: Rename columns
- 6d: Output to intermediate_file_6

In [17]:
# Step 6: Dropping and renaming columns

# 6a: Load data
df = pd.read_csv(intermediate_file_5)

# 6b: Dropping Columns
df = df.iloc[:,[1,2,3,4,5]]

# 6c: Renaming Columns        
# Rename a column (e.g., rename 'Name' to 'last_name')
df.columns = ['email', 'name', 'date_of_birth', 'id_number', 'phone_number']

# 6d: Output to intermediate_file_6
df.to_csv(intermediate_file_6, index=False, encoding='utf-8')

C:\Users\max_m\AppData\Local\Temp\ipykernel_28456\2913616572.py:4: DtypeWarning: Columns (0,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(intermediate_file_5)


# STEP 7: DEALING WITH INVALID DATA
- 7a: Load data
- 7b: Validate and correct emails
- 7c: Output to intermediate_file_7

In [18]:
# Step 7: Dealing with invalid data
# 7a: Load data
def correct_email_domain(email):
    """
    Corrects common typos in email domains.
    """
    if not isinstance(email, str):
        return email  # Return the input as-is if not a string
    email = email.lower()
    domain_corrections = {
        "hotmial.com": "hotmail.com",
        "hotmil.com": "hotmail.com",
        "hotnail.com": "hotmail.com",
        "hotmiall.com": "hotmail.com",
        "hotmial.ocm": "hotmail.com",
        "hocmail.com": "hotmail.com",
        "gnail.com": "gmail.com",
        "yaho.com": "yahoo.com",
        "outllok.com": "outlook.com",
    }
    for typo, correct in domain_corrections.items():
        if typo in email:
            return email.replace(typo, correct)
    return email

# Validate Email Function
def validate_email(email):
    """
    Validates email addresses after correcting typos.
    """
    if not isinstance(email, str):  # Ensure email is a string
        return False, email  # Invalid if not a string
    email = correct_email_domain(email)
    # Corrected regex pattern
    pattern = r"^[a-zA-Z0-9!#$%&'*+/=?^_{|}~.-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$"
    return bool(re.match(pattern, email)), email

# Main Validation and Correction Function
def validate_and_correct_emails(input_file, output_file, garbage_file):
    """
    Validates and corrects emails, saving valid and invalid data separately.
    """
    df = pd.read_csv(input_file, encoding="utf-8-sig")
    garbage = pd.DataFrame()

    # Apply email validation and correction
    df["is_valid_email"], df["email"] = zip(*df["email"].apply(lambda x: validate_email(x)))

    # Handle invalid emails
    invalid_emails = df[~df["is_valid_email"]].copy()
    invalid_emails["Issue"] = "Invalid email"

    # Save invalid emails to the garbage file
    garbage = pd.concat([garbage, invalid_emails], ignore_index=True)
    garbage.to_csv(garbage_file, index=False, encoding="utf-8-sig")

    # Keep only valid emails in the original DataFrame
    df = df[df["is_valid_email"]].drop(columns=["is_valid_email"])
    df.to_csv(output_file, index=False, encoding="utf-8-sig")

logger.info("Validating and correcting email addresses")
validate_and_correct_emails(intermediate_file_6, intermediate_file_7, garbage_file)



2025-02-13 14:02:12,928 - INFO - Validating and correcting email addresses
2025-02-13 14:02:12,928 - INFO - Validating and correcting email addresses


# STEP 8: DEALING WITH INVALID DATA
- 8a: Load data
- 8b: Validate and correct mobile_phone
- 8c: Output to intermediate_file_8

In [19]:
# Step 8:
# 8a: Load data
df = pd.read_csv(intermediate_file_7)

# 8b: Clean Phone Numbers
def clean_phone_number(phone_number):
    """
    Cleans the phone_number field:
    1. Leaves empty phone_number fields unchanged.
    2. Strips non-numeric characters (e.g., letters, +, -, (, ), *, spaces, etc.).
    3. Ensures valid phone numbers have lengths between 7 and 15 digits.
       Invalid numbers are returned as blank.
    """
    if pd.isnull(phone_number) or phone_number == "":
        # Leave empty fields unchanged
        return ""

    # Convert to string for processing
    phone_number = str(phone_number)

    # Strip all non-numeric characters
    cleaned_number = re.sub(r"\D", "", phone_number)  # Retains only digits (0-9)

    # Validate phone numbers length (7-15 characters)
    if 7 <= len(cleaned_number) <= 15:
        return cleaned_number
    else:
        # Return blank for invalid phone numbers
        return ""

logger.info("Cleaning phone numbers")

# Ensure the column exists before cleaning
if "phone_number" in df.columns:
    # Apply the cleaning function to the 'phone_number' column
    df["phone_number"] = df["phone_number"].apply(clean_phone_number)
else:
    logger.warning("'phone_number' column is missing in the input file.")

# 8c: Output to intermediate_file_8
df.to_csv(intermediate_file_8, index=False, encoding="utf-8")


2025-02-13 14:04:01,425 - INFO - Cleaning phone numbers
2025-02-13 14:04:01,425 - INFO - Cleaning phone numbers


# STEP 9: DROPPING INSUFFICIENT DATA ROWS
- 9a: Load data
- 9b: Drop rows with insufficient data
- 9c: Output to intermediate_file_9

In [20]:
# Step 9: Dealing with Invalid Data

# 9a: Load data
df = pd.read_csv(intermediate_file_8)

# 9b: Remove rows where 'email', 'name' or 'id_number' are blank
df_cleaned = df.replace('', None).dropna(subset=['email', 'name', 'id_number'])

# 9c: Output to intermediate_file_9
df_cleaned.to_csv(intermediate_file_9, index=False, encoding='utf-8')



# STEP 10: REMOVE DUPLICATE RECORDS
- 10a: Load data
- 10b: Drop duplicate records
- 10c: Put phone number back to int64
- 10d: Output to final cleaned data file

In [21]:
# Step 10: Remove duplicate files

# 10a: Load data
df = pd.read_csv(intermediate_file_9)
garbage = pd.read_csv(garbage_file)

# 10b: Remove duplicate rows
df = df.replace('', None).dropna(subset=['email', 'name', 'id_number'])

# 10b: Check for duplicate emails
logger.info("Checking for duplicate email addresses")
duplicates = df[df.duplicated(subset=["email"], keep="first")]
duplicates["Issue"] = "duplicate email"
garbage = pd.concat([garbage, duplicates], ignore_index=True)
    
# Remove duplicates from the cleaned data
df = df.drop_duplicates(subset=["email"], keep="first")

# 10c: Ensure phone_number is treated as a string to avoid decimals in the CSV
if 'phone_number' in df.columns:
    df['phone_number'] = df['phone_number'].astype('Int64')


# 10d: Output to final cleaned file
df.to_csv(cleaned_file, index=False, encoding='utf-8')


2025-02-13 14:07:02,410 - INFO - Checking for duplicate email addresses
2025-02-13 14:07:02,410 - INFO - Checking for duplicate email addresses
C:\Users\max_m\AppData\Local\Temp\ipykernel_28456\2505399779.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicates["Issue"] = "duplicate email"


# STEP 11: FINAL REPORT AND CLEANUP
- 11a: Print statistics
- 11b: Delete intermediate files

In [22]:
# Step 11: Final report and cleanup

# 11a: Final cleaned data preview
print("\nCleaned Data:")
print(df_cleaned.head())

# 11b: Delete intermediate files
# List of intermediate files to be cleaned up
intermediate_files = [
    intermediate_file_1,
    intermediate_file_2,
    intermediate_file_3,
    intermediate_file_4,
    intermediate_file_5,
    intermediate_file_6,
    intermediate_file_7,
    intermediate_file_8,
    intermediate_file_9,
    intermediate_file_10
]

for file in intermediate_files:
    if os.path.exists(file):
        os.remove(file)
        logger.info(f"Deleted intermediate file: {file}")
    else:
        logger.warning(f"Intermediate file not found: {file}")



2025-02-13 14:07:31,686 - WARNING - Intermediate file not found: ./datasets/processed/step1/intermediate_1.csv
2025-02-13 14:07:31,686 - WARNING - Intermediate file not found: ./datasets/processed/step1/intermediate_1.csv
2025-02-13 14:07:31,691 - WARNING - Intermediate file not found: ./datasets/processed/step2/intermediate_2.csv
2025-02-13 14:07:31,691 - WARNING - Intermediate file not found: ./datasets/processed/step2/intermediate_2.csv
2025-02-13 14:07:31,694 - WARNING - Intermediate file not found: ./datasets/processed/step3/intermediate_3.csv
2025-02-13 14:07:31,694 - WARNING - Intermediate file not found: ./datasets/processed/step3/intermediate_3.csv
2025-02-13 14:07:31,697 - WARNING - Intermediate file not found: ./datasets/processed/step4/intermediate_4.csv
2025-02-13 14:07:31,697 - WARNING - Intermediate file not found: ./datasets/processed/step4/intermediate_4.csv
2025-02-13 14:07:31,800 - INFO - Deleted intermediate file: ./datasets/processed/step5/intermediate_5.csv
2025-0


Cleaned Data:
                        email name date_of_birth           id_number  \
0       shipeijie_001@163.com  史培杰    1984-03-01  370902198403264812   
1   liuchunhua770512@yahoo.cn  刘春华    1977-05-01  65010319770512604x   
2  zhaojing830803@hotmail.com   赵静    1983-08-01  421002198308031881   
3      shiming586@hotmail.com  夏世明    1979-02-01  362222197902253512   
4             bossjj2@163.com  朱泽宇    1986-06-05  320683198606014314   

   phone_number  
0  113774254415  
1  113482701254  
2  113764095160  
3  113381756335  
4  115301612043  


2025-02-13 14:07:31,854 - INFO - Deleted intermediate file: ./datasets/processed/step6/intermediate_6.csv
2025-02-13 14:07:31,901 - INFO - Deleted intermediate file: ./datasets/processed/step7/intermediate_7.csv
2025-02-13 14:07:31,901 - INFO - Deleted intermediate file: ./datasets/processed/step7/intermediate_7.csv
2025-02-13 14:07:31,953 - INFO - Deleted intermediate file: ./datasets/processed/step8/intermediate_8.csv
2025-02-13 14:07:31,953 - INFO - Deleted intermediate file: ./datasets/processed/step8/intermediate_8.csv
2025-02-13 14:07:32,001 - INFO - Deleted intermediate file: ./datasets/processed/step9/intermediate_9.csv
2025-02-13 14:07:32,001 - INFO - Deleted intermediate file: ./datasets/processed/step9/intermediate_9.csv
2025-02-13 14:07:32,006 - WARNING - Intermediate file not found: ./datasets/processed/step10/intermediate_10.csv
2025-02-13 14:07:32,006 - WARNING - Intermediate file not found: ./datasets/processed/step10/intermediate_10.csv
